# **Извлечение именованных сущностей(ФИО, организация, локация, дата, сумма денег)**

In [1]:
!python -m spacy download ru_core_news_md

2022-11-16 15:56:45.190437: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 11.2 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=5720ea5f6a152a35b95fb62ee33f98ab37c57f445328a19cceb313e9d3a0815c
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [2]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 70 kB/s 
     |████████████████████████████████| 41 kB 119 kB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26118 sha256=a35408697e4630193f415c1600ccf237b9ed972a2ce543e6547e961be0058fb4
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)
import ru_core_news_md

In [4]:
test = ['''
В Кировский районный суд г. Ростова-на-Дону
г. Ростов-на-Дону, пер. Газетный, 36
Кривошеева Михаила Ивановича
г. Ростов-на-Дону, переулок Журавлева 1, кв. 5а

Заявление

14 февраля 2019 года я подал иск о взыскании с Куролесова Григория Макаровича 72 тыс. рублей заемных средств. В соответствии с действующим законодательством причитающаяся мне к уплате госпошлина составляет 2360 руб.*, но в настоящее время я не имею возможности уплатить эту сумму в связи со сложным финансовым положением – уволен в связи с ликвидацией фирмы, нахожусь в поиске работы.

На основании изложенного, руководствуясь ст. 333.41 НК РФ, прошу отсрочить уплату государственной пошлины по моему иску к Куролесову Г.М. до окончания рассмотрения дела.

Приложение: справка Центра занятости.

14.02.19

Кривошеев М.И.

*Расчет госпошлины осуществляется согласно ст. 333.19 НК РФ. По иску от 20001 до 100000 руб. госпошлина 800 руб. плюс 3% от суммы свыше 20000 руб. 72000-20000= 52000 руб. 52000 *3% = 1560 руб. 1560 + 800 = 2360 руб.
''',
"""
В Первомайский районный суд
Тамбовской области
Заявитель: Самосвалов Леонид Васильевич,
адрес: 478547, Тамбовская область,
Первомайский район, пос. Первомайский,
ул. Сиреневая, д. 136
Заинтересованное лицо:
Нотариус Светозарова Нина Семеновна,
478547, Тамбовская область,
Первомайский район, пос. Первомайский,
ул. Сиреневая, д.198,
ИНН 1346987497
Заявление об отказе в совершении нотариального действия
20 ноября 2021 г. я обратился к Светозаровой Нине Семеновне, частному нотариусу, по адресу нотариальной конторы: пос. Первомайский, ул. Сиреневая, д. 198, с просьбой нотариального удостоверения договора ренты с Анисимовой Светланой Сергеевной.

Нотариусом в этом мне было отказано со ссылкой на п. 5 ст. 48 Основ законодательства о нотариате, поскольку данная сделка не соответствует требованиям закона и подлежит государственной регистрации. Постановление об отказе совершить нотариальное действие № 5 я получил 24.11.2020 года.

Отказ нотариуса считаю необоснованным, так как Федеральным законом от 30.12.2012 г. № 302-ФЗ правила государственной регистрации таких договоров после 01.03.2013 г. не применяются.

В соответствии со ст. 310 ГПК РФ,

Прошу:

Обязать нотариуса Светозарову Нину Семеновну удостоверить договор ренты с Анисимовой Светланой Сергеевной.

Приложение:

Квитанция об уплате госпошлины в суд
Постановление нотариуса от 24.11.2021 г. № 5
Договор ренты от 19.11.2021 г. с приложениями
Уведомление об отправке документов и иска заинтересованному лицу
28.11.2021 Самосвалов Л.В.
""",
"""
Председателю Дзержинского районного суда
г. Новосибирска
Новосибирской области
Заявитель: Гранина Вероника Михайловна,
адрес: 630019, г. Новосибирск,
пер. Промышленный, д. 8, кв. 17
Жалоба на судью
В производстве Дзержинского районного суда г. Новосибирска Новосибирской области находится гражданское дело № 2-6847/2017 о праве собственности на самовольную постройку к Администрации г. Новосибирска. Я являюсь в данном деле Истцом. Исковое заявление подано мной 15 августа 2017 г.

В ходе рассмотрения дела судьей Потаповой Ниной Григорьевной неоднократно допускались нарушения процессуальных норм и требований. В частности, согласно определению о подготовке дела к рассмотрению на Ответчика была возложена обязанность предоставить отзыв на исковое заявление. Ни на предварительном заседании 14.09.2017 г., ни в основное заседание от 10.10.2017 г. отзыв предоставлен не был. При этом Ответчик надлежащим образом извещался, о чем имеются соответствующие отметки в деле, но в заседания его представитель не явился. Судьей рассмотрение дела перенесено на 08.11.2017 г. и опять отложено.

В качестве доказательства по гражданскому делу мною было предоставлено заключение строительно-технической экспертизы, которую я проводила до обращения в суд за свой счет. Несмотря на это, судья несколько раз предложила мне ходатайствовать о назначении судебной экспертизы по гражданскому делу (за мой счет). Несмотря на то, что согласно ст. 79, ч. 2 ст. 96 ГПК РФ суд вправе самостоятельно назначить экспертизу.

Согласно ст. 3 Закона РФ от 26 июня 1992 г. № 3132-I «О статусе судей в Российской Федерации» судья при исполнении своих полномочий, а также во внеслужебных отношениях должен избегать всего, что могло бы умалить авторитет судебной власти, достоинство судьи или вызвать сомнение в его объективности, справедливости и беспристрастности.

Согласно ст. 11 Кодекса судейской этики от 19 декабря 2012 г. судья должен добросовестно исполнять свои профессиональные обязанности и принимать все необходимые меры для своевременного рассмотрения дел и материалов. В соответствии со ст. 61 ГПК РФ разбирательство дел в судах осуществляется в сроки, установленные ГПК; продление этих сроков допустимо в случаях и в порядке, которые установлены ГПК РФ, но судопроизводство должно осуществляться в разумный срок.

Статьей 154 ГПК РФ установлено, что гражданские дела рассматриваются и разрешаются судом до истечения двух месяцев со дня поступления заявления в суд. Действиями судьи по затягиванию процесса нарушаются мои права на судебную защиту.

На основании изложенного, прошу принять меры к устранению нарушений действующего законодательства РФ, допущенных судьей Потаповой Н.Г.

10.11.2017 г. В.М. Гранина
""",
"""
В Российский союз
автостраховщиков
115093, г. Москва, ул. Люсиновская, д. 27, строение 3
Руководителю Управления
Федеральной антимонопольной службы
Республики Хакассия (копия)
665018, г. Абакан,
ул. Клубная, 192
Руководителю Управления
Роспотребнадзора по
Республике Хакассия (копия)
665023, г. Абакан, ул. Строителей, 17
от Варнакова Петра Сергеевича,
адрес: 665014, г. Абакан, пер. Профсоюзный,
10-7
тел. 846011465845
Жалоба на страховую компанию
15.10.2016 г. с заявлением о заключении договора ОСАГО я обратился в страховую компанию «Витта», офис которой расположен по адресу: г. Абакан, ул. 1-я Восточная, 140 г. К заявлению о заключении договора я предоставил полный пакет документов, в том числе диагностическую карту автотранспортного средства, документы на автомобиль и устанавливающие личность. Прием проводил менеджер Константинова А.С.

Однако в заключении договора мне в нарушении ст. 426 Гражданского кодекса РФ мне было отказано. Менеджер сослалась на необходимость предоставить транспортное средство для техосмотра в страховую компанию. На мое заявление о том, что техосмотр я уже прошел и в соответствии с Законом «Об ОСАГО» имею право пройти техосмотр по месту своего жительства.

Менеджер сообщила, что они могут принять диагностическую карту только при условии заключения с ними договора добровольного страхования жизни. В противном случае полис ОСАГО мне выдан не будет. Я пригласил начальника отдела по страхованию клиентов Попова А.А., он внимательно выслушал, но повторил требование об обязательности добровольного (!) страхования.

Подобные действия страховой компании «Витта» противоречат Закону. В соответствии со ст. 1 Закона «Об ОСАГО», ст. 16 Закона о защите прав потребителей, Положения Банка от 19.09.2014 н. № 431-П (п. 1.5. Приложения 1 О правилах страхования гражданской ответственности». Поскольку указанная страховая компания является фактически монополистом на рынке, я не могу исполнить свою обязанность и пользоваться автотранспортным средством.

Прошу провести проверку изложенных в жалобе фактов, привлечь ответственных лиц к ответственности, а также обязать страховую компанию «Витта» заключить договор ОСАГО и выдать страховой полис. 20.10.2016 г. ной направлена претензия по ОСАГО, в случае неудовлетворения которой я вынужден буду обратиться в суд с исковым заявлением о защите прав потребителя.

21.10.2016 г. Варнаков П.С.
""",
"""
В Зейский районный суд
Амурской области
Заявитель: Рахмонов Иван Александрович,
с. Сосновый бор, ул. Мухина, 211
телефон: 89111002534
Заинтересованное лицо:
Зайцев Александр Васильевич,
11.05.1981 г.р., место рождения: пос. Тягун,
с. Сосновый бор, ул. Слонова, 112,
ИНН 5976656854
телефон: 89111002533
Заявление о признании вещи бесхозяйной
17.12.2020 г. я на основании договора купли-продажи приобрел у Зайцева Александра Васильевича, 11.05.1981 г.р., земельный участок по адресу: с. Сосновый бор, пер. Мирный, д. 479, с кадастровым номером 48:895:568, а также здание на нем. Объекты недвижимости переданы мне по Акту приема-передачи от 18.12.2019 г. Право собственности на земельный участок и здание зарегистрированы в Управлении Росреестра по Амурской области 20.12.2019 г.

При осмотре участка после регистрации права мной обнаружен горизонтально-фрезерный станок 6К81Г, который в предмет договора не входил. И по вышеуказанному Акту приема-передачи не передавался. 25.12.2019 г. написал письмо продавцу — Зайцеву А.В. — с просьбой забрать станок, но 09.01.2020 г. получил ответ, что данный станок ему не нужен. Таким образом, с 17.12.2019 г. я фактически владею указанным станком.

В соответствии со ст. 226 ГК РФ брошенные вещи поступают в собственность лица, вступившего во владение ими, если по заявлению этого лица они признаны судом бесхозяйными.

На основании изложенного, руководствуясь ст. 226 ГК РФ, ст. 290-291 ГПК РФ,

Прошу:

Признать горизонтально-фрезерный станок 6К81Г, 2017 года выпуска, бесхозяйной вещью, передать его в мою собственность.
Приложение:

Квитанция об оплате госпошлины в суд
Копия договора купли-продажи № 1243 от 17.12.2019 г.
Выписка из ЕГРН о переходе права собственности
Копия уведомления Зайцеву А.В. и ответ
Акт осмотра земельного участка
Уведомление об отправке документов и заявления заинтересованному лицу
15.01.2021 Рахмонов И.А.
""",
"""
В Октябрьский районный суд г. Барнаула
Заявитель: Поломонов Роман Иванович,
адрес: 179000, г. Москва, ул. Сизова, д. 789,
Заинтересованное лицо: ООО «Строительная инициатива»,
656154, г. Барнаул, ул. Васильковая, д. 41,
ИНН 354684698749611
ЗАЯВЛЕНИЕ О ВОССТАНОВЛЕНИИ ПРАВ ПО УТРАЧЕННОЙ ЦЕННОЙ БУМАГЕ
Мне, Поломонову Роману Ивановичу, ООО «Строительная инициатива» выдан жилищный сертификат на предъявителя (дата сертификата: 11 января 2020 г., № 78924-ФН) сроком действия до 15 января 2022 г., общий объем эмиссии сертификатов: 132. Жилищным сертификатом предоставлено право его предъявителю заключить договор купли-продажи на приобретение квартиры общей площадью 25 кв.м. у ООО «Строительная инициатива». Цена сертификата составила 500 000 (пятьсот тысяч).

Контролировать целевое использование денежных средств по сертификату должен банк ЗАО «ЮТБ». В то же время, указанный жилищный сертификат был утрачен мной: 12.02.2021 г. у меня из личного автомобиля во дворе дома похищена сумка с документами, о чем в отделе полиции мне выдана соответствующая справка о возбуждении уголовного дела.

Руководствуясь статьями 262-263, 294 — 295 ГПК РФ, статьей 148 ГК РФ,

Прошу:

Запретить ООО «Строительная инициатива» производить по жилищному сертификату от 11.01.2020 г. № 78924-ФН платежи или выдачу.
Признать жилищный сертификат на предъявителя от 11.01.2020 г. № 78924-ФН, выданный ООО «Строительная инициатива», недействительным.
Восстановить мои права заявителя по жилищному сертификату и обязать ООО «Строительная инициатива» выдать Поломонову Роману Ивановичу новый жилищный сертификат взамен утраченного.
Приложения:

Квитанция об уплате госпошлины в суд
Квитанция об оплате номинальной стоимости жилищного сертификата
Справка отдела полиции о возбуждении уголовного дела по факту кражи
Уведомление об отправке документов заинтересованному лицу
Поломонов Р.И. 18.05.2021 г.
""",
"""
В Красногвардейский районный суд Белгородской области
Истец: Завгород Лилия Васильевна
адрес: 309920, г. Бирюч,
ул. Васильева, д. 25, кв. 71,
Ответчик: Малахов Виктор Юрьевич,
05.09.1992 г.р., место рождения: г. Витебск,
адрес: 309921, г. Бирюч, ул. Светлая, 13,
паспорт 1744 № 646251365,
выдан 04.08.2015 ОФМС г.Бирюч
в рамках дела № 1-234/2022
о взыскании материального ущерба
Ходатайство об истребовании доказательств
В производстве Красногвардейского районного суда находится гражданское дело № 1-234/2022 по иску к Малахову Виктору Юрьевичу о возмещении материального ущерба. Исковые требования связаны с заключением 15.11.2021 г. договора арены автотранспортного средства, по условиям которого во владение Ответчика поступил автомобиль OPEL ASTRA, 2008 г.в., гос. номер А521РА 51.

18.04.2022 г. Ответчик не выбрал безопасную скорость для совершения маневра и допустил опрокидывание автотранспортного средства. У истца имеется копия акта о ДТП от 18.04.2022 г., оригинал и постановление об отказе в возбуждении дела об административном правонарушении находятся в МО МВД по Красненскому району, на подведомственной территории и произошло ДТП.

Одним из юридически значимых обстоятельств в рассматриваемом гражданском деле является доказательство факта причинения вреда имуществу в результате виновных действий Ответчика. В соответствии с определением вышеуказанного суда от 16.05.2022 г. о принятии искового заявления к производству и ст. 56 ГПК РФ бремя доказывания указанного обстоятельства возложено на истца.

Самостоятельно представить в суд акт о ДТП и постановление об отказе в возбуждении дела об административном правонарушении не представляется возможным. Предоставление указанных документов по моему запросу запрещает ФЗ «О персональных данных». Приобщение указанных материалов необходимо для доказательства вины Ответчика, факта ДТП, что подтверждает обоснованность исковых требований и позицию истца.

Указанные документы находятся в МО МВД «Красненский» Белгородской области.

На основании изложенного, руководствуясь ст. 57 ГПК РФ,

Прошу:

Запросить в МО МВД «Красненский» Белгородской области следующие доказательства: акт о ДТП от 18.04.2022 г. с участием Малахова Виктора Юрьевича.
Истребованные доказательства приобщить к материалам гражданского дела 1-234/2022 по иску Завгород Л.В. к Малахову В.Ю. о возмещении материального ущерба.
Приложение:

Уведомление о вручении копии ходатайства участникам дела
03.06.2022 г. Завгород Л.В.
""",
"""
В Ардатовский районный суд
Нижегородской области
Заявитель: Шипунова Анастасия Михайловна,
адрес: 607130, г. Ардатов, ул. Коммунистическая, 84-41,
Должник: ООО «Страхование»,
адрес: 603950, г. Нижний Новгород, пр. Московский, 19,
ИНН 87964165416541
в рамках дела № 2-181/2021
о защите прав потребителя
Заявление об индексации присужденных сумм
Решением Ардатовского районного суда Нижегородской области от 17.09.2021 г. удовлетворены требования Шипуновой Анастасии Михайловны к ООО «Страхование», г. Нижний Новгород, о взыскании денежных средств в размере 159 000 руб. по иску о защите прав потребителя, компенсации морального вреда в размере 15 000 руб. Решение вступило в законную силу после рассмотрения апелляционной жалобы 19.12.2021 г.

Исполнительный лист вместе с заявлением о возбуждении исполнительного производства заявитель (взыскатель) предъявил в отдел судебных приставов Ленинского района г. Нижний Новгород. 25.12.2021 г. возбуждено исполнительное производство №289-484/2021.

Решение Ардатовского районного суда от 17.09.2021 г. Ответчик исполнил только 19.05.2022 г. То есть спустя 6 месяцев после вступления его в законную силу и 8 месяцев с даты вынесения. В результате длительного неисполнения решения суда денежные средства, присужденные мне, обесценились.

Договором между взыскателем и должником предусмотрена индексация долга любой из Сторон в зависимости от роста потребительских цен (п. 8.14 договора). В соответствии со ст. 208 ГПК РФ, официальными данными Федеральной службы государственной статистики индекс роста потребительских цен составил:

октябрь – 100,44,
ноябрь – 100,35
декабрь – 100,26
январь – 101
февраль – 100,56
март – 100,44
апрель – 100,4
Таким образом, расчет индексации

159000+15000 (основная сумма долга) = 174000*1,0044*1,0035*1,0026*1,01*1,0056*1,0044*1,004 -174000= 6089,37 руб.

На основании изложенного, руководствуясь ст. 208 ГПК РФ,

Прошу:

Взыскать с ООО «Страхование» в мою пользу денежные средства в размере 6 089,37 руб. в качестве индексации присужденной решением Ардатовского районного суда Нижегородской области от 17.09.2021 г. денежной суммы за период с 17 сентября 2021 г. и по 19.05.2022 г.
Приложение:

Расчет индексации
Сведения об индексе потребительских цен в РФ (размещенный на официальном сайте)
Постановление судебного пристава-исполнителя об окончании исполнительного производства
Копия договора страхования
Уведомление об отправке заявления и документов должнику
20.06.2022 г. Шипунова А.М.
""",
"""
В Ленинградский областной суд
Истец: Сигачева Егора Ильича,
адрес: 188410, Ленинградская область,
г. Волосово, ул. Вингиссара, дом 109, кв.521,
тел. 89000000018
Ответчик: Головлев Александр Титович,
адрес: 188410, Ленинградская область,
г. Волосово, ул. Вингиссара, дом 109, кв.525,
тел. 89000000019
ЧАСТНАЯ ЖАЛОБА НА ОПРЕДЕЛЕНИЕ СУДА
по гражданскому делу № 2-1354/2021
16 октября 2021 года Волосовским районным судом Ленинградской области вынесено определение по ходатайству о прекращении производства по гражданскому делу по иску Сигачева Е.И. к Головлеву А.Т. по иску о возмещении ущерба от залива квартиры. В обоснование определения суд указал, что в производстве суда уже имелось гражданское дело № 2-354/2020. В котором истцом заявлялись такие же исковые требования по аналогичным основаниям.

Считаю определение суда незаконным. Согласно ст. 220 ГПК РФ обязательным условием прекращения производства по делу по аналогичным предмету и основаниям иска является совпадение и сторон, то есть истца и ответчика. В гражданском деле № 2-354/2020 я не участвовал. Исковое заявление предъявляла в суд моя жена Сигачева А.В. Впоследствии она отказалась от иска, поскольку в ходе рассмотрения дела выяснилось, что она не является участником спорных правоотношений. И ее права и законные интересы действиями ответчика нарушены не были.

Суд указанное положение закона не учел, в связи с чем допустил ошибку, которая препятствует мне получить возмещение причиненного ответчиком ущерба.

Руководствуясь статьями 331-333 ГПК РФ,

Прошу:

Отменить определение Волосовского районного суда Ленинградской области от 16 октября 2021 года о прекращении производства по гражданскому делу № 2-1354/2021 по иску Сигачева Е.И. к Головлеву А.Т. о возмещении ущерба от залива квартиры и направить гражданское дело для рассмотрения по существу в суд первой инстанции.

Рассмотреть частную жалобу без моего участия в суде.

Перечень прилагаемых к жалобе документов:

Копия частной жалобы – 1 экз.

20.10.2021 г. Подпись Сигачев
""",
"""
В Электростальский городской суд
Истец: Заяц Мирослава Марковна
проживающая по адресу:
144006, Московская область,
г. Электросталь, ул. Даля, д. 2568 кв. 999
тел. 82255245548
Ответчик: УК «Недомашний уют»
юридический адрес: 144007, Московская область,
г. Электросталь, ул. Малины, д. 255 оф. 257,
ИНН 687964168432
тел. 82369852147
Исковое заявление на управляющую компанию
С 2020 года я в соответствии с договором дарения от 26.01.2020 года являюсь собственником квартиры № 999, расположенной по адресу: Московская область, г. Электросталь, ул. Даля, д. 2568, общей площадью 56 кв.м. 22.01.2020 года я и ответчик заключили письменное соглашение по управлению многоквартирным домом № 2568 по ул. Даля.

На основании Жилищного кодекса РФ, управляющая организация должна предоставлять услуги по содержанию дома, его общего имущества и в полном объеме осуществлять другую деятельность, связанную с управлением дома. Согласно Закону РФ «О защите прав потребителей» ответчик, поставляя коммунальные услуги, должен предоставить потребителю услуги качества, соответствующего условиям договора, требованиям стандартов, санитарным требованиям и установленным нормам. Предоставляемые услуги должны быть безопасными для жизни, здоровья, окружающей среды, не причиняющие вреда имуществу.

Я добросовестно исполняю все обязательства по договору, что подтверждается копиями квитанций о внесении платы за жилье и коммунальные услуги. Ответчик не в полной мере осуществляет свои обязанности. В частности, нарушаются санитарно-гигиенические нормы: в квартире течет ржавая вода; из подвала в квартиру поступает неприятный запах, что свидетельствует о засорении систем вентиляции; не осуществляется уборка общих мест пользования; придомовая территория убирается нерегулярно.

Я неоднократно отсылал письменные претензии в адрес ответчика, но никаких мер не принято. Последняя претензия поступила в УК 22.11.2021 года, но осталась без ответа.

Руководствуясь статьей 162 ЖК РФ, статьями 17, 29 Закона РФ «О защите прав потребителей»

Прошу:

Обязать Управляющую компанию «Недомашний уют» безвозмездно устранить недостатки в обслуживании дома по адресу: Московская область, г. Электросталь, ул. Даля д. 2568, а именно:
устранить неисправности в системе водопровода;
устранить неисправности в дренажной системе и вентиляции дома;
производить уборку, мойку пола и лестничных пролетов один раз в неделю;
устранить неприятный запах из подвала.
Приложение:

Договор с управляющей компанией
Копии претензий, направленных ответчику
Копия ответов на претензии
Квитанции об оплате коммунальных услуг
Выписка из ЕГРН на квартиру
Акт об осмотре мест общего пользования, составленный собственниками квартир дома № 2658
Уведомление об отправке документов ответчику
30.03.2022 Заяц М.М.
""",
"""
В Головинский районный суд г. Москвы
Истец: УК«Василек»
адрес: 125131, г. Москва, ул. Ключевая, д. 421, кор. 2,
ИНН 4769165165
тел. 82334598452
Представитель истца по доверенности:
Газин Эдуард Моисеевич,
адрес для корреспонденции:
125132, г. Москва, ул. Спартака, д. 257, кв. 954
тел. 82478547854
Ответчик: Водопьянова Нелли Николаевна,
13.03.1981 г.р., место рождения: г. Урюпинск,
адрес: 125133, г. Москва, ул. Краснопольская,
д. 401, кв. 1005,
паспорт 1713 № 69896416
Цена иска: 70 214 руб. 57 коп.
Исковое заявление о взыскании оплаты за жилье и коммунальные услуги
По результатам голосования собственников жилья многоквартирного дома № 401, расположенного по адресу: г. Москва, ул. Краснопольская, согласно протоколу голосования №1 от 23.06.2020 года, был определен способ управления многоквартирным домом – управляющей компанией. Управляющей компанией выбрана УК «Василек».

Ответчик является единоличным собственником квартиры № 1005, входящей в состав указанного дома. Общая площадь квартиры составляет 91,8 кв.м.

Жилищный кодекс РФ обязывает граждан к своевременной и полной оплате за жилье и коммунальные услуги. Кодексом предусмотрено, что плата должна перечисляться до 10 числа следующего за отчертным месяца. Жильцы, несвоевременно или частично оплатившие жилье и коммунальные услуги, обязаны оплатить пеню за каждый день просрочки.

В результате ненадлежащего исполнения Водопьяновой Н.Н. обязанностей по внесению оплаты за жилье и коммунальные услуги, предусмотренные договором № 1005 от 29.06.2020 года, за период с 10.07.2020 по 02.03.2021 за ответчицей образовался долг в сумме 70 214,57 руб., состоящий из оплаты:

работ по капитальному ремонту дома – 25473,03 руб.;
за ремонт и содержание дома – 15245,56 руб.;
за коммунальные услуги – 29495,98 руб., а именно: теплоснабжение – 12803,14 руб.; электроснабжение – 11619,20 руб.; — водоснабжение – 5073,64 руб.
Расчет взыскиваемой задолженности прилагается к исковому заявлению.

Истцом неоднократно направлялись претензии должнику. Последняя вручена 25.02.2021 года. До настоящего времени долг не погашен.

Нарушение порядка расчетов влечет затруднительность своевременного расчета с поставщиками коммунальных услуг и подрядчиками, обслуживающими дом, дополнительными затратами на оплату пени, а для всех жильцов, в том числе и добросовестных, — приостановлением или прекращением поставки таких услуг.

Судебный приказ мирового судьи судебного участка № 13 Головинского района г. Москвы от 20.02.2021 г. отменен по заявлению ответчика.

Руководствуясь статьями 153, 155 ЖК РФ, статьей 309 ГК РФ, статьями 98, 100, 131-132 ГПК РФ.,

Прошу:

Взыскать с Водопьяновой Нелли Николаевны задолженность по оплате жилья и коммунальных услуг за период с 10.07.2020 по 10.02.2021 в размере 70214 руб. 57 коп.
Взыскать с Водопьяновой Нелли Николаевны сумму уплаченной госпошлины в размере 2906 руб. 42 коп.
Приложение:

Квитанция госпошлины в суд
Устав УК «Василек»
Свидетельство ОГРН и ИНН истца
Копия договора № 1005 от 29.06.2020 года
Копия лицевого счета Водопьяновой Н.Н.
Расчет задолженности
Копии претензий от 15.09.2020, 20.10.2020, 14.01.2021, 25.02.2021
Копия доверенности на представление интересов ЖСК «Василек»
Определение об отмене судебного приказа
Уведомление об отправке документов и иска ответчику
10.03.2021 года Представитель истца Газин Эдуард Моисеевич
""",
"""
В Головинский районный суд г. Москвы
Истец: УК«Василек»
адрес: 125131, г. Москва, ул. Ключевая, д. 421, кор. 2,
ИНН 4769165165
тел. 82334598452
Представитель истца по доверенности:
Газин Эдуард Моисеевич,
адрес для корреспонденции:
125132, г. Москва, ул. Спартака, д. 257, кв. 954
тел. 82478547854
Ответчик: Водопьянова Нелли Николаевна,
13.03.1981 г.р., место рождения: г. Урюпинск,
адрес: 125133, г. Москва, ул. Краснопольская,
д. 401, кв. 1005,
паспорт 1713 № 69896416
Цена иска: 70 214 руб. 57 коп.
Исковое заявление о взыскании оплаты за жилье и коммунальные услуги
По результатам голосования собственников жилья многоквартирного дома № 401, расположенного по адресу: г. Москва, ул. Краснопольская, согласно протоколу голосования №1 от 23.06.2020 года, был определен способ управления многоквартирным домом – управляющей компанией. Управляющей компанией выбрана УК «Василек».

Ответчик является единоличным собственником квартиры № 1005, входящей в состав указанного дома. Общая площадь квартиры составляет 91,8 кв.м.

Жилищный кодекс РФ обязывает граждан к своевременной и полной оплате за жилье и коммунальные услуги. Кодексом предусмотрено, что плата должна перечисляться до 10 числа следующего за отчертным месяца. Жильцы, несвоевременно или частично оплатившие жилье и коммунальные услуги, обязаны оплатить пеню за каждый день просрочки.

В результате ненадлежащего исполнения Водопьяновой Н.Н. обязанностей по внесению оплаты за жилье и коммунальные услуги, предусмотренные договором № 1005 от 29.06.2020 года, за период с 10.07.2020 по 02.03.2021 за ответчицей образовался долг в сумме 70 214,57 руб., состоящий из оплаты:

работ по капитальному ремонту дома – 25473,03 руб.;
за ремонт и содержание дома – 15245,56 руб.;
за коммунальные услуги – 29495,98 руб., а именно: теплоснабжение – 12803,14 руб.; электроснабжение – 11619,20 руб.; — водоснабжение – 5073,64 руб.
Расчет взыскиваемой задолженности прилагается к исковому заявлению.

Истцом неоднократно направлялись претензии должнику. Последняя вручена 25.02.2021 года. До настоящего времени долг не погашен.

Нарушение порядка расчетов влечет затруднительность своевременного расчета с поставщиками коммунальных услуг и подрядчиками, обслуживающими дом, дополнительными затратами на оплату пени, а для всех жильцов, в том числе и добросовестных, — приостановлением или прекращением поставки таких услуг.

Судебный приказ мирового судьи судебного участка № 13 Головинского района г. Москвы от 20.02.2021 г. отменен по заявлению ответчика.

Руководствуясь статьями 153, 155 ЖК РФ, статьей 309 ГК РФ, статьями 98, 100, 131-132 ГПК РФ.,

Прошу:

Взыскать с Водопьяновой Нелли Николаевны задолженность по оплате жилья и коммунальных услуг за период с 10.07.2020 по 10.02.2021 в размере 70214 руб. 57 коп.
Взыскать с Водопьяновой Нелли Николаевны сумму уплаченной госпошлины в размере 2906 руб. 42 коп.
Приложение:

Квитанция госпошлины в суд
Устав УК «Василек»
Свидетельство ОГРН и ИНН истца
Копия договора № 1005 от 29.06.2020 года
Копия лицевого счета Водопьяновой Н.Н.
Расчет задолженности
Копии претензий от 15.09.2020, 20.10.2020, 14.01.2021, 25.02.2021
Копия доверенности на представление интересов ЖСК «Василек»
Определение об отмене судебного приказа
Уведомление об отправке документов и иска ответчику
10.03.2021 года Представитель истца Газин Эдуард Моисеевич
""",
"""
В Гагаринский районный суд г. Москвы
Истец: Кактусов Петр Петрович,
адрес: 119052, г. Москва,
пер. Солнечный, д. 752, кв. 1111
тел. 89615786542
Ответчик: Борисова Роза Юрьевна,
17.02.1992 г.р., место рождения не известно,
адрес: 119051, г. Москва, ул. Радужная,
д. 859, кв. 1233
тел. 89632589651,
паспорт 5555 № 364861
Исковое заявление о расторжении договора найма жилого помещения
Согласно договору дарения № 23561245 от 11.06.2013 года истец, Кактусов П.П., является собственником квартиры № 233, расположенной по адресу: г. Москва, ул. Радужная, д.859. Право собственности подтверждается Свидетельством о регистрации права серии 48 78 номер 86946 от 15.07.2013 года.

20.12.2020 года я сдал внаем квартиру Борисовой Р.Ю. на шесть лет. По договору найма жилого помещения ответчица, в том числе, взяла на себя обязанность вносить плату за жилье на мой расчетный счет в банке в размере 10 000 руб. Ежемесячно до 30 числа каждого месяца (п. 4.2. Договора).

Первые полгода платежи были регулярными и претензий к Борисовой Р.Ю. у меня не возникало. После ответчик произвел ремонт и перепланировку жилого помещения. Без предварительного согласования со мной. Кроме того, ответчик перестал вносить установленную договором оплату. В телефонном разговоре Борисова Р.Ю. сообщила, что ремонт произведен в счет оплаты жилья. И следующий платеж будет внесен после того, как затраченная на ремонт сумма будет зачтена в счет платежей.

С доводами Борисовой Р.Ю. я категорически не согласен. 20.12.2021 г. я направил ответчику письмо, в котором потребовал досрочно расторгнуть договор и освободить нанимаемую квартиру. Ответа на требование я не получил. Своими действиями Борисова Р.Ю. нарушила условия, предусмотренные договором найма.

Руководствуясь статьями 8, 12, 687 Гражданского Кодекса РФ, статьями 131-132 ГПК РФ,

Прошу:

Расторгнуть договор найма жилого помещения по адресу: г. Москва, ул. Радужная, д. 859, кв. 1233, от 20.12.2020 г.
Выселить Борисову Розу Юрьевну из занимаемой квартиры по адресу: г. Москва, ул. Радужная, д. 859, кв. 1233.
Приложения:

Квитанция госпошлины в суд
Договор дарения (копия)
Копия свидетельства о регистрации права собственности
Договора найма жилого помещения от 20.12.2020 г.
Банковская выписка счета Кактусова П.П.
Копия требования от 20.12.2021 года
Ходатайство о вызове свидетелей
Уведомление об отправке документов ответчику
15.05.2022 Кактусов П.П.
"""]

## **Для извлечения ФИ0(PER), организаций(ORG) и локаций(LOC) используем библиотеку SpaCy**

In [15]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)

nlp = ru_core_news_md.load()

In [16]:
for text in test:
  result = nlp(text)
  text = text.translate({ord(i): None for i in '«' and '»'})
  data = list(dates_extractor(text))
  money = list(money_extractor(text))
  print(text)
  print("\n")
  for named_entity in result.ents:
    print(named_entity, named_entity.label_)
  print(data)
  print(money)
  print("\n\n\n")


В Кировский районный суд г. Ростова-на-Дону
г. Ростов-на-Дону, пер. Газетный, 36
Кривошеева Михаила Ивановича
г. Ростов-на-Дону, переулок Журавлева 1, кв. 5а

Заявление

14 февраля 2019 года я подал иск о взыскании с Куролесова Григория Макаровича 72 тыс. рублей заемных средств. В соответствии с действующим законодательством причитающаяся мне к уплате госпошлина составляет 2360 руб.*, но в настоящее время я не имею возможности уплатить эту сумму в связи со сложным финансовым положением – уволен в связи с ликвидацией фирмы, нахожусь в поиске работы.

На основании изложенного, руководствуясь ст. 333.41 НК РФ, прошу отсрочить уплату государственной пошлины по моему иску к Куролесову Г.М. до окончания рассмотрения дела.

Приложение: справка Центра занятости.

14.02.19

Кривошеев М.И.

*Расчет госпошлины осуществляется согласно ст. 333.19 НК РФ. По иску от 20001 до 100000 руб. госпошлина 800 руб. плюс 3% от суммы свыше 20000 руб. 72000-20000= 52000 руб. 52000 *3% = 1560 руб. 1560 + 800 = 2

## **Извлечение суммы денег и дат можно выполнить через библиотеку natasha**